In [1]:
import numpy as np
from math import cos, sin
import matplotlib.pyplot as plt
import imageio

In [2]:
# vetor de estados do robot no instante t: Xt = [xt, yt, tetat, vt]
# xt e yt -> posição x-y 2d do robot
# tetat -> orientação
# vt -> velocidade

# vetor de entrada no instante t: ut = [vt, omegat]
# omegat -> yaw rate (taxa de movimento angular sobre o eixo vertical), medida pelo giroscópio

# vetor de observação no instante t: zt = [xt, yt]

# modelo dinâmico: x[t+1] = F * xt + B * ut

In [3]:
# Globals
# Estimation params for EKF
Q = np.diag([0.1, 0.1, np.deg2rad(1.0), 1.0]) ** 2
R = np.diag([1.0, 1.0]) ** 2

# Sim params
gps_noise = np.diag([0.5, 0.5]) ** 2
input_noise = np.diag([1.0, np.deg2rad(30.0)]) ** 2

dt = 0.1
sim_time = 50.0

In [4]:
def u(v=1.0, yaw=0.1):
    """
    Devolve o vetor de entrada
    num dado instante.
    """
    return np.array([[v, yaw]]).T

In [5]:
def mm(x, u, dt=0.1):
    """
    Devolve o valor de x[t+1]
    de acordo com o modelo dinâmico
    definido na ficha 4:
    xt = F * xt + B * ut
    """

    F = np.identity(4, dtype=float)
    F[3, 3] = 0
    B = np.array([[dt * cos(x[2, 0]), 0],
                  [dt * sin(x[2, 0]), 0],
                  [0.0, dt],
                  [1.0, 0.0]])

    return F.dot(x) + B.dot(u)

In [6]:
def observation(x, xdr, u):
    x = mm(x, u)

    # add noise to gps x-y
    zx = x[0, 0] + np.random.randn() * gps_noise[0, 0]
    zy = x[1, 0] + np.random.randn() * gps_noise[1, 1]
    z = np.array([[zx, zy]])

    # add noise to input
    ud1 = u[0, 0] + np.random.randn() * input_noise[0, 0]
    ud2 = u[1, 0] + np.random.randn() * input_noise[1, 1]
    ud = np.array([[ud1, ud2]]).T

    xdr = mm(xdr, ud)

    return x, z, xdr, ud

In [7]:
def jF(x, u):
    """
    Devolve a matriz jacobiana do modelo
    dinâmico da ficha 4.
    """
    yaw = x[2, 0]
    v = u[0, 0]
    return np.array([
        [1.0, 0.0, -dt * v * np.sin(yaw), dt * np.cos(yaw)],
        [0.0, 1.0, dt * v * np.cos(yaw), dt * np.sin(yaw)],
        [0.0, 0.0, 1.0, 0.0],
        [0.0, 0.0, 0.0, 1.0]])

In [8]:
def jH():
    """
    Devolve a matriz jacobiana do modelo de
    observação da ficha 4.
    """
    return np.identity(4)[0:2]

In [9]:
def om(x):
    """
    Devolve o modelo de observação
    de acordo com o enunciado da ficha 4:
    zt = H * xt
    """
    return jH().dot(x)

In [10]:
def ekf(x, p, z, u):
    """
    Estimação da localização
    do robot através do
    Extended Kalman Filter
    """
    # Predict
    xpred = mm(x, u)
    jf = jF(xpred, u)
    ppred = jf.dot(p).dot(jf.T) + Q

    # Update
    jh = jH()
    zpred = om(xpred)
    y = z.T - zpred
    S = jh.dot(ppred).dot(jh.T) + R
    K = ppred.dot(jh.T).dot(np.linalg.inv(S))
    x = xpred + K.dot(y)
    p = (np.eye(len(x)) - K.dot(jh)).dot(ppred)

    return x, p

In [11]:
def save(hxpred, xpred, hxtrue, xtrue, hxdr, xdr, hz, z):
    hxpred = np.hstack((hxpred, xpred))
    hxtrue = np.hstack((hxtrue, xtrue))
    hxdr = np.hstack((hxdr, xdr))
    hz = np.vstack((hz, z))
    return hxpred, hxtrue, hxdr, hz

In [12]:
%matplotlib inline

def main():
    print('Initiated robot sim!')
    
    time = 0.

    # vetor de estados [x, y, yaw, v]
    xpred = np.zeros((4, 1))    
    xtrue = np.zeros((4, 1))    
    ppred = np.eye(4)

    # dead reckoning
    xdr = np.zeros((4, 1))

    # histórico
    hxpred = xpred
    hxtrue = xtrue
    hxdr = xtrue
    hz = np.zeros((1, 2))

    fig = plt.figure()
    fig.show()
    
    # for gif
    imgs = []

    while time <= sim_time:
        time += dt

        xtrue, z, xdr, ud = observation(xtrue, xdr, u())
        xpred, ppred = ekf(xpred, ppred, z, ud)

        # guardar no histórico
        hxpred, hxtrue, hxdr, hz = save(
            hxpred, xpred, hxtrue, xtrue,
            hxdr, xdr, hz, z
        )

        plt.cla()
        plt.plot(hz[:, 0], hz[:, 1], ".g")
        plt.plot(hxtrue[0, :].flatten(),
                    hxtrue[1, :].flatten(), "-b")
        plt.plot(hxdr[0, :].flatten(),
                    hxdr[1, :].flatten(), "-k")
        plt.plot(hxpred[0, :].flatten(),
                    hxpred[1, :].flatten(), "-r")
        plt.axis("equal")
        plt.grid(True)

        # gif
        # filename = f'{time * 1/dt}.jpeg'
        # imgs.append(f'./frames/{filename}')
        # plt.savefig(filename)

        plt.pause(0.001)

In [13]:
# Descomentar linha abaixo para gerar imagens
# main()
# ⚠ Ver resultados no ficheiro gifs/ekf-final.gif